<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fbase-models/models/base_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path
import shutil
import time
import psutil

# Regenerar el código con las condiciones específicas
notebook_globals = {
    "USE_CROSS_VALIDATION": False,
    "ENABLED_MODELS": ['CNN', 'GRU'],
    "ENABLED_EXPERIMENTS": ['time+cycles', 'all_features'],
    "ENABLED_HORIZONS": [3],
}

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn ace_tools
    BASE_PATH = '/content/drive/MyDrive/ml_precipitation_prediction'
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

print(f"Entorno configurado. Usando ruta base: {BASE_PATH}")

# Si BASE_PATH viene como string, lo convertimos
BASE_PATH = Path(BASE_PATH)

# Ahora puedes concatenar correctamente
data_output_dir = BASE_PATH / 'data' / 'output'
model_output_dir = BASE_PATH / 'models' / 'output'


Mounted at /content/drive
Cloning into 'ml_precipitation_prediction'...
remote: Enumerating objects: 880, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 880 (delta 8), reused 6 (delta 6), pack-reused 863 (from 1)
Receiving objects: 100% (880/880), 99.80 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (476/476), done.
/content/ml_precipitation_prediction
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.7 MB/s eta 0:00:00
Entorno configurado. Usando ruta base: /content/drive/MyDrive/ml_precipitation_prediction


In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input

# ==== Variables de control ====
USE_CROSS_VALIDATION = True
ENABLED_MODELS = ['CNN', 'GRU']
ENABLED_EXPERIMENTS = ['time+cycles', 'all_features']
ENABLED_HORIZONS = [3]
input_window = 96  # 8 años (mensual)

# ==== Configuración de entorno ====
print("🔍 Detectando dispositivo disponible...")
gpu_devices = tf.config.list_physical_devices('GPU')
USE_GPU = bool(gpu_devices)
if USE_GPU:
    print("✅ GPU detectada:", gpu_devices[0].name)
    print("ℹ️ Entrenamiento acelerado con GPU activado.")
else:
    print("⚠️ No se detectó GPU. Usando CPU.")

# ==== Funciones auxiliares ====
def build_model(model_type, input_shape, output_neurons):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM, GRU, Bidirectional, Reshape
    model = Sequential()
    model.add(Input(shape=input_shape, dtype='float32'))
    if model_type == 'LSTM':
        model.add(LSTM(64))
    elif model_type == 'GRU':
        model.add(GRU(64))
    elif model_type == 'BLSTM':
        model.add(Bidirectional(LSTM(64)))
    elif model_type == 'CNN':
        model.add(Reshape((*input_shape, 1)))
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
    model.add(Dense(output_neurons))
    model.compile(optimizer='adam', loss='mse')
    return model

def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-5))) * 100
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, mape, r2

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size=16):
        self.x, self.y = x_set.astype(np.float32), y_set.astype(np.float32)
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

def to_dataset(x, y):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    return tf.data.Dataset.from_tensor_slices((x, y)).batch(16).prefetch(tf.data.AUTOTUNE)

# ==== Directorios y Dataset ====
print("📂 Configurando directorios y cargando dataset...")
try:
    model_output_dir_STH = model_output_dir / 'ST_HybridWaveStack'
    curves_dir = model_output_dir_STH / "learning_curves"
    if not model_output_dir_STH.exists():
        model_output_dir_STH.mkdir(parents=True)
    if not curves_dir.exists():
        curves_dir.mkdir(parents=True)
    print(f"✔️ Modelos en: {model_output_dir_STH}")
    print(f"✔️ Curvas en: {curves_dir}")

    file_path = data_output_dir / "complete_dataset_with_features_with_clusters_elevation_with_windows.nc"
    ds = xr.open_dataset(file_path)
    print(f"✔️ Dataset cargado desde: {file_path}")
except Exception as e:
    raise RuntimeError(f"❌ Error cargando dataset o creando carpetas: {e}")

# ==== Configuración de experimentos ====
experiment_settings = {
    "time+cycles": ['year','month','month_sin','month_cos','doy_sin','doy_cos'],
    "time+cycles+lag": ['year','month','month_sin','month_cos','doy_sin','doy_cos',
        'total_precipitation_lag1','total_precipitation_lag2','total_precipitation_lag3',
        'total_precipitation_lag4','total_precipitation_lag12','total_precipitation_lag24','total_precipitation_lag36'],
    "time+cycles+lag+elev": ['year','month','month_sin','month_cos','doy_sin','doy_cos',
        'total_precipitation_lag1','total_precipitation_lag2','total_precipitation_lag3',
        'total_precipitation_lag4','total_precipitation_lag12','total_precipitation_lag24','total_precipitation_lag36',
        'elevation','slope','aspect'],
    "all_features": ['year','month','month_sin','month_cos','doy_sin','doy_cos',
        'total_precipitation_lag1','total_precipitation_lag2','total_precipitation_lag3',
        'total_precipitation_lag4','total_precipitation_lag12','total_precipitation_lag24','total_precipitation_lag36',
        'elevation','slope','aspect','cluster_elevation']
}

# Validación de variables
ds_vars = set(ds.data_vars)
for name, vars_list in experiment_settings.items():
    missing = [v for v in vars_list if v not in ds_vars]
    if missing:
        raise ValueError(f"❌ Faltan vars para '{name}': {missing}")
print("✅ Variables requeridas presentes.")

results = []

# ==== Entrenamiento modular ====
for exp_name, variables in experiment_settings.items():
    if exp_name not in ENABLED_EXPERIMENTS:
        continue

    print(f"\n🚀 Experimento: {exp_name}")
    try:
        # Preparar datos
        cluster_idx = variables.index('cluster_elevation') if 'cluster_elevation' in variables else None
        subset = ds[variables].to_array().transpose('time','latitude','longitude','variable').values
        if cluster_idx is not None:
            cd = subset[...,cluster_idx]
            subset[...,cluster_idx] = LabelEncoder().fit_transform(cd.ravel()).reshape(cd.shape)
        subset = subset.astype(np.float32)
        target = ds['total_precipitation'].values
        samples,lat,lon,feats = subset.shape
        X = subset.reshape(samples, lat*lon, feats)
        y = target.reshape(samples, lat*lon)
        mask = ~np.isnan(y)
        X, y = X[mask], y[mask]

        # Generar secuencias
        X_seq, Y_targets = [], {h:[] for h in ENABLED_HORIZONS}
        for i in range(len(X) - input_window - max(ENABLED_HORIZONS)):
            X_seq.append(X[i:i+input_window])
            for h in ENABLED_HORIZONS:
                Y_targets[h].append(y[i+input_window+h-1])
        X_seq = np.array(X_seq)
        Y_targets = {h: np.array(Y_targets[h]) for h in ENABLED_HORIZONS}

        # --- FILTRAR Secuencias con NaNs y reportar ---
        def filtrar_secuencias(Xs, ys):
            total = len(Xs)
            valid = (~np.isnan(Xs).any(axis=(1,2))) & (~np.isnan(ys))
            kept = valid.sum()
            lost = total - kept
            pct = 100 * lost/total
            print(f"📉 Secuencias orig.: {total}, válidas: {kept}, eliminadas: {lost} ({pct:.2f}%)")
            return Xs[valid], ys[valid]

        for h in ENABLED_HORIZONS:
            X_seq, Y_targets[h] = filtrar_secuencias(X_seq, Y_targets[h])

        if len(X_seq)==0:
            print(f"⚠️ No quedan secuencias válidas para '{exp_name}'. Saltando.")
            continue

        input_shape = (X_seq.shape[1], X_seq.shape[2])

        # Cross-validation
        for model_name in ENABLED_MODELS:
            print(f"🔧 Modelo: {model_name}")
            kf = KFold(n_splits=3, shuffle=False)
            for h in ENABLED_HORIZONS:
                fold=1
                for tr_idx,va_idx in kf.split(X_seq):
                    X_tr, X_va = X_seq[tr_idx], X_seq[va_idx]
                    y_tr, y_va = Y_targets[h][tr_idx], Y_targets[h][va_idx]
                    # Escalar
                    scalerX=StandardScaler()
                    shpX = X_tr.shape
                    X_tr = scalerX.fit_transform(X_tr.reshape(-1,shpX[-1])).reshape(shpX)
                    shpX2 = X_va.shape
                    X_va = scalerX.transform(X_va.reshape(-1,shpX[-1])).reshape(shpX2)
                    scalery=StandardScaler()
                    y_tr = scalery.fit_transform(y_tr.reshape(-1,1)).reshape(y_tr.shape)
                    y_va = scalery.transform(y_va.reshape(-1,1)).reshape(y_va.shape)

                    # Data generators
                    train_gen = DataGenerator(X_tr, y_tr, batch_size=16)
                    val_gen   = DataGenerator(X_va, y_va, batch_size=16)

                    model_path = model_output_dir_STH / f"{exp_name.replace('+','_')}_{model_name}_H{h}_F{fold}.h5"
                    if model_path.exists():
                        print(f"⏩ Ya existe fold {fold}, skip.")
                        fold+=1
                        continue

                    model = build_model(model_name, input_shape, 1)
                    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
                    history = model.fit(train_gen,
                                        validation_data=val_gen,
                                        epochs=20,
                                        verbose=1,
                                        callbacks=[es])

                    # Evaluación
                    y_pred = model.predict(X_va).flatten()
                    y_true = y_va.flatten()
                    rmse, mae, mape, r2 = evaluate(y_true,y_pred)
                    results.append({
                        'experiment': exp_name,'model': model_name,
                        'horizon':h,'fold':fold,
                        'RMSE':rmse,'MAE':mae,'MAPE':mape,'R2':r2,
                        'epochs':len(history.history['loss'])
                    })

                    # Curva
                    plt.figure()
                    plt.plot(history.history['loss'], label='Train')
                    plt.plot(history.history['val_loss'], label='Val')
                    plt.title(f'{exp_name}-{model_name}-H{h}-F{fold}')
                    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
                    plt.savefig(curves_dir / f"{exp_name.replace('+','_')}_{model_name}_H{h}_F{fold}.png")
                    plt.close()

                    model.save(model_path)
                    print(f"💾 Guardado fold {fold}")
                    fold+=1

    except Exception as e:
        print(f"❌ Error en '{exp_name}': {e}")

# ==== Guardar resultados ====
results_df = pd.DataFrame(results)
results_df.to_csv("resultados_modelos_cv_8anios_mvp.csv", index=False)
print(results_df.head())

import ace_tools_open as tools
tools.display_dataframe_to_user(name="Resultados CV MVP", dataframe=results_df)
print("✅ Proceso finalizado con éxito.")


🔍 Detectando dispositivo disponible...
✅ GPU detectada: /physical_device:GPU:0
ℹ️ Entrenamiento acelerado con GPU activado.
📂 Configurando directorios y cargando dataset...
✔️ Modelos en: /content/drive/MyDrive/ml_precipitation_prediction/models/output/ST_HybridWaveStack
✔️ Curvas en: /content/drive/MyDrive/ml_precipitation_prediction/models/output/ST_HybridWaveStack/learning_curves
✔️ Dataset cargado desde: /content/drive/MyDrive/ml_precipitation_prediction/data/output/complete_dataset_with_features_with_clusters_elevation_with_windows.nc
✅ Variables requeridas presentes.

🚀 Experimento: time+cycles
📉 Secuencias orig.: 2101351, válidas: 2101351, eliminadas: 0 (0.00%)
🔧 Modelo: CNN
⏩ Ya existe fold 1, skip.
⏩ Ya existe fold 2, skip.
⏩ Ya existe fold 3, skip.
🔧 Modelo: GRU
⏩ Ya existe fold 1, skip.
⏩ Ya existe fold 2, skip.
⏩ Ya existe fold 3, skip.

🚀 Experimento: all_features
📉 Secuencias orig.: 2101351, válidas: 1958611, eliminadas: 142740 (6.79%)
🔧 Modelo: CNN
⏩ Ya existe fold 1, sk

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81609/81609 ━━━━━━━━━━━━━━━━━━━━ 224s 3ms/step - loss: 0.3210 - val_loss: 0.2857
Epoch 2/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 221s 3ms/step - loss: 0.2829 - val_loss: 0.2821
Epoch 3/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 220s 3ms/step - loss: 0.2829 - val_loss: 0.2867
Epoch 4/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 221s 3ms/step - loss: 0.2816 - val_loss: 0.2683
Epoch 5/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 221s 3ms/step - loss: 0.2805 - val_loss: 0.2793
Epoch 6/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 221s 3ms/step - loss: 0.2792 - val_loss: 0.3289
Epoch 7/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 222s 3ms/step - loss: 0.2796 - val_loss: 0.2920
Epoch 8/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 222s 3ms/step - loss: 0.2766 - val_loss: 0.2990
Epoch 9/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 221s 3ms/step - loss: 0.2796 - val_loss: 0.2787
20403/20403 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step


💾 Guardado fold 2
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81609/81609 ━━━━━━━━━━━━━━━━━━━━ 227s 3ms/step - loss: 0.2896 - val_loss: 0.4083
Epoch 2/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 225s 3ms/step - loss: 0.2516 - val_loss: 0.4463
Epoch 3/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 224s 3ms/step - loss: 0.2478 - val_loss: 0.5073
Epoch 4/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 225s 3ms/step - loss: 0.2497 - val_loss: 0.4159
Epoch 5/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 224s 3ms/step - loss: 0.2475 - val_loss: 0.4630
Epoch 6/20
81609/81609 ━━━━━━━━━━━━━━━━━━━━ 226s 3ms/step - loss: 0.2457 - val_loss: 0.5432
20403/20403 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step


💾 Guardado fold 3
🔧 Modelo: GRU
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43050/81609 ━━━━━━━━━━━━━━━━━━━━ 3:37 6ms/step - loss: 0.3263